## Notebook to handle NFI data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../scripts/')
import numpy as np
import matplotlib.pyplot as plt
import rasterio, rasterio.plot
import xarray as xr
import rioxarray as rxr
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import loadpaths
import land_cover_analysis as lca
import land_cover_visualisation as lcv
import land_cover_models as lcm
from torch.utils.data import TensorDataset, DataLoader

path_dict = loadpaths.loadpaths()

/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
path_nfi_full = '/home/tplas/data/gis/National_Forest_Inventory_Woodland_England_2020/National_Forest_Inventory_Woodland_England_2020.shp'

In [6]:
pols_nfi = lca.load_pols(path_nfi_full)
print(f'Loaded {len(pols_nfi)} polygons from {path_nfi_full}')

Loaded 355973 polygons from /home/tplas/data/gis/National_Forest_Inventory_Woodland_England_2020/National_Forest_Inventory_Woodland_England_2020.shp


In [8]:
outline_pd = lca.load_pols(path_dict['pd_outline'])
outline_pd = outline_pd.iloc[0]['geometry']

In [19]:
df_pols_nfi_pd = pols_nfi.intersection(outline_pd)

/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/pygeos/set_operations.py:129: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


In [36]:
## Find non empty polygons in pd series:
inds_inside_pd = df_pols_nfi_pd.area > 0
df_nfi_pd = pols_nfi[inds_inside_pd]
df_nfi_pd = df_nfi_pd.reset_index(drop=True)
# df_pols_nfi_pd = df_pols_nfi_pd.reset_index(drop=True)
print(f'Found {np.sum(inds_inside_pd)} polygons in PD')

Found 3018 polygons in PD


In [48]:
# df_nfi_pd['geometry'].area - df_nfi_pd['Area_ha'] * 10000

In [38]:
print(df_nfi_pd['IFT_IOA'].unique())

['Bare area' 'Grassland' 'Quarry' 'Urban' 'Road' 'Other vegetation'
 'Open water' 'Assumed woodland' 'Broadleaved' 'Conifer' 'Felled'
 'Ground prep' 'Low density' 'Mixed mainly broadleaved'
 'Mixed mainly conifer' 'Young trees' 'Coppice' 'Shrub' 'Windblow'
 'Uncertain']


In [39]:
mapping_dict_nfi = {'Broadleaved': 'C1', 
                    'Conifer': 'C2',
                    'Felled': 'C5',
                    'Mixed mainly broadleaved': 'C3',
                    'Mixed mainly conifer': 'C3',
                    'Shrub': 'C4',
                    'Low density': 'C4'
                    # 'Young trees': 'C5'  # leaving out young trees because can be any category 
                    }

In [40]:
df_nfi_pd['Class_low'] = df_nfi_pd['IFT_IOA'].map(mapping_dict_nfi)
## Remove polygons with no LC:
df_nfi_pd = df_nfi_pd[df_nfi_pd['Class_low'].notna()] 
df_nfi_pd = df_nfi_pd.reset_index(drop=True)
print(f'Found {len(df_nfi_pd)} polygons with LC')

Found 2597 polygons with LC


In [49]:
df_nfi_pd.to_file('/home/tplas/repos/cnn-land-cover/content/NFI_data/NFI_pd.shp')